In [ ]:
!pip install pydub

In [ ]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import layers, regularizers, optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten, Bidirectional,  MultiHeadAttention, Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import matplotlib.pyplot as plt
from pydub import AudioSegment
from tensorflow.keras.layers.experimental import preprocessing


In [ ]:
# Function to load data from folder
def load_data_from_folder(folder):
    labels = []
    data = []

    for subdir, _, files in os.walk(folder):
        for file in files:
            if file.endswith(".npy"):
                file_path = os.path.join(subdir, file)
                label = os.path.basename(subdir)  # Use the parent folder name as the label
                labels.append(label)
                data.append(np.load(file_path))

    return np.array(data), np.array(labels)

# Specify the root folder containing subdirectories with .npy files
root_folder = "/content/drive/MyDrive/MFCC_Features"

# Load data from subdirectories
data, labels = load_data_from_folder(root_folder)

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)
encoded_labels = to_categorical(encoded_labels, num_classes=num_classes)

# Split the data into training and testing sets with stratification
X_train, X_test, y_train, y_test = train_test_split(
    data, encoded_labels, test_size=0.2, random_state=42, stratify=encoded_labels
)

# Data normalization using TensorFlow preprocessing layers
normalization_layer = preprocessing.Normalization()
normalization_layer.adapt(X_train)  # Compute mean and variance based on training data
X_train_normalized = normalization_layer(X_train)
X_test_normalized = normalization_layer(X_test)

# Data augmentation using TensorFlow preprocessing layers
data_augmentation = Sequential([
    preprocessing.Rescaling(1./255),  # Normalize pixel values to [0,1]
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
])

X_train_augmented = data_augmentation(X_train_normalized)
X_test_augmented = data_augmentation(X_test_normalized)

# Compute class weights for handling class imbalance
class_weights = compute_class_weight("balanced", classes=np.unique(encoded_labels.argmax(axis=1)), y=encoded_labels.argmax(axis=1))
class_weight_dict = dict(zip(np.unique(encoded_labels.argmax(axis=1)), class_weights))


STARTING MFFC Feature TRAINING FROM HERE

In [ ]:
# Build the LSTM model with added layers for better performance
model = Sequential()
model.add(LSTM(64, input_shape=(X_train_augmented.shape[1], X_train_augmented.shape[2])))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Use the Adam optimizer for better convergence
optimizer = Adam(learning_rate=0.0001)

# Compile the model with categorical_crossentropy loss and accuracy metric
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model with a more expressive variable name for clarity
epochs = 10
batch_size = 22
validation_data = (X_test_augmented, y_test)

history = model.fit(X_train_augmented, y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

Epoch 1/10
49/49 [==============================] - 6s 36ms/step - loss: 2.6989 - accuracy: 0.4086 - val_loss: 2.6868 - val_accuracy: 0.4345
Epoch 2/10
49/49 [==============================] - 1s 20ms/step - loss: 2.6693 - accuracy: 0.4339 - val_loss: 2.6391 - val_accuracy: 0.4345
Epoch 3/10
49/49 [==============================] - 1s 19ms/step - loss: 2.5637 - accuracy: 0.4339 - val_loss: 2.3713 - val_accuracy: 0.4345
Epoch 4/10
49/49 [==============================] - 1s 19ms/step - loss: 2.2138 - accuracy: 0.4339 - val_loss: 2.0989 - val_accuracy: 0.4345
Epoch 5/10
49/49 [==============================] - 1s 13ms/step - loss: 2.1442 - accuracy: 0.4339 - val_loss: 2.0897 - val_accuracy: 0.4345
Epoch 6/10
49/49 [==============================] - 1s 13ms/step - loss: 2.1285 - accuracy: 0.4339 - val_loss: 2.0846 - val_accuracy: 0.4345
Epoch 7/10
49/49 [==============================] - 1s 13ms/step - loss: 2.1494 - accuracy: 0.4321 - val_loss: 2.0841 - val_accuracy: 0.4345
Epoch 8/10
49

In [ ]:
# Build the LSTM model with added bidirectional layers for better performance
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train_augmented.shape[1], X_train_augmented.shape[2])))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Use the Adam optimizer for better convergence
optimizer = Adam(learning_rate=0.001)

# Compile the model with categorical_crossentropy loss and accuracy metric
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model with a more expressive variable name for clarity
epochs = 10
batch_size = 32
validation_data = (X_test_augmented, y_test)

history = model.fit(X_train_augmented, y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data)


Epoch 1/10
34/34 [==============================] - 13s 93ms/step - loss: 2.4202 - accuracy: 0.4189 - val_loss: 2.1788 - val_accuracy: 0.4345
Epoch 2/10
34/34 [==============================] - 1s 41ms/step - loss: 2.1618 - accuracy: 0.4339 - val_loss: 2.0779 - val_accuracy: 0.4345
Epoch 3/10
34/34 [==============================] - 1s 40ms/step - loss: 2.1327 - accuracy: 0.4339 - val_loss: 2.0740 - val_accuracy: 0.4345
Epoch 4/10
34/34 [==============================] - 1s 41ms/step - loss: 2.1305 - accuracy: 0.4339 - val_loss: 2.0745 - val_accuracy: 0.4345
Epoch 5/10
34/34 [==============================] - 2s 73ms/step - loss: 2.1219 - accuracy: 0.4339 - val_loss: 2.0842 - val_accuracy: 0.4345
Epoch 6/10
34/34 [==============================] - 3s 75ms/step - loss: 2.1122 - accuracy: 0.4339 - val_loss: 2.0913 - val_accuracy: 0.4345
Epoch 7/10
34/34 [==============================] - 3s 76ms/step - loss: 2.1061 - accuracy: 0.4339 - val_loss: 2.0990 - val_accuracy: 0.4345
Epoch 8/10
3

In [ ]:
#Changing the Dropout
# Build the LSTM model with added bidirectional layers for better performance
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train_augmented.shape[1], X_train_augmented.shape[2])))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

# Use the Adam optimizer for better convergence
optimizer = Adam(learning_rate=0.001)

# Compile the model with categorical_crossentropy loss and accuracy metric
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model with a more expressive variable name for clarity
epochs = 10
batch_size = 32
validation_data = (X_test_augmented, y_test)

history = model.fit(X_train_augmented, y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data)


Epoch 1/10
34/34 [==============================] - 14s 89ms/step - loss: 2.3617 - accuracy: 0.4142 - val_loss: 2.1452 - val_accuracy: 0.4157
Epoch 2/10
34/34 [==============================] - 1s 40ms/step - loss: 2.0948 - accuracy: 0.4386 - val_loss: 2.1455 - val_accuracy: 0.4157
Epoch 3/10
34/34 [==============================] - 1s 40ms/step - loss: 2.0913 - accuracy: 0.4386 - val_loss: 2.1419 - val_accuracy: 0.4157
Epoch 4/10
34/34 [==============================] - 1s 40ms/step - loss: 2.0967 - accuracy: 0.4386 - val_loss: 2.1388 - val_accuracy: 0.4157
Epoch 5/10
34/34 [==============================] - 1s 40ms/step - loss: 2.0967 - accuracy: 0.4386 - val_loss: 2.1418 - val_accuracy: 0.4157
Epoch 6/10
34/34 [==============================] - 2s 55ms/step - loss: 2.0890 - accuracy: 0.4386 - val_loss: 2.1459 - val_accuracy: 0.4157
Epoch 7/10
34/34 [==============================] - 2s 69ms/step - loss: 2.0897 - accuracy: 0.4386 - val_loss: 2.1356 - val_accuracy: 0.4157
Epoch 8/10
3

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Bidirectional, Dense, Dropout, LSTM, Attention
from tensorflow.keras.optimizers import Adam

# Input layer
input_layer = Input(shape=(X_train_augmented.shape[1], X_train_augmented.shape[2]))

# Bidirectional LSTM with attention
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(input_layer)
attention = Attention()([lstm_layer, lstm_layer])
dropout_1 = Dropout(0.5)(attention)

lstm_layer_2 = Bidirectional(LSTM(64))(dropout_1)
attention_2 = Attention()([lstm_layer_2, lstm_layer_2])
dropout_2 = Dropout(0.5)(attention_2)

# Dense layers
dense_1 = Dense(128, activation='relu')(dropout_2)
dropout_3 = Dropout(0.2)(dense_1)

output_layer = Dense(num_classes, activation='softmax')(dropout_3)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Use the Adam optimizer for better convergence
optimizer = Adam(learning_rate=0.001)

# Compile the model with categorical_crossentropy loss and accuracy metric
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 32
validation_data = (X_test_augmented, y_test)

history = model.fit(X_train_augmented, y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data)


Epoch 1/10
34/34 [==============================] - 15s 95ms/step - loss: 2.3684 - accuracy: 0.4180 - val_loss: 2.0951 - val_accuracy: 0.4345
Epoch 2/10
34/34 [==============================] - 1s 43ms/step - loss: 2.1158 - accuracy: 0.4339 - val_loss: 2.0817 - val_accuracy: 0.4345
Epoch 3/10
34/34 [==============================] - 1s 42ms/step - loss: 2.0826 - accuracy: 0.4339 - val_loss: 2.0838 - val_accuracy: 0.4345
Epoch 4/10
34/34 [==============================] - 1s 43ms/step - loss: 2.1151 - accuracy: 0.4339 - val_loss: 2.1019 - val_accuracy: 0.4345
Epoch 5/10
34/34 [==============================] - 1s 44ms/step - loss: 2.0969 - accuracy: 0.4339 - val_loss: 2.0784 - val_accuracy: 0.4345
Epoch 6/10
34/34 [==============================] - 2s 56ms/step - loss: 2.0989 - accuracy: 0.4339 - val_loss: 2.0913 - val_accuracy: 0.4345
Epoch 7/10
34/34 [==============================] - 2s 72ms/step - loss: 2.0999 - accuracy: 0.4339 - val_loss: 2.0744 - val_accuracy: 0.4345
Epoch 8/10
3

In [ ]:
# Build the model
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Use the Adam optimizer for better convergence
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 32
validation_data = (X_test, y_test)

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

# Evaluate the model on the test set
_, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

# Print the combined accuracy result
print(f"Combined Accuracy on Test Set: {test_accuracy * 100:.2f}%")

Epoch 1/10
34/34 [==============================] - 13s 78ms/step - loss: 2.2576 - accuracy: 0.3889 - val_loss: 2.0876 - val_accuracy: 0.4345
Epoch 2/10
34/34 [==============================] - 1s 23ms/step - loss: 2.0946 - accuracy: 0.4321 - val_loss: 2.0533 - val_accuracy: 0.4345
Epoch 3/10
34/34 [==============================] - 1s 24ms/step - loss: 2.0517 - accuracy: 0.4358 - val_loss: 2.0481 - val_accuracy: 0.4345
Epoch 4/10
34/34 [==============================] - 1s 23ms/step - loss: 2.0131 - accuracy: 0.4330 - val_loss: 2.0340 - val_accuracy: 0.4345
Epoch 5/10
34/34 [==============================] - 1s 23ms/step - loss: 1.9681 - accuracy: 0.4358 - val_loss: 2.0880 - val_accuracy: 0.4157
Epoch 6/10
34/34 [==============================] - 1s 24ms/step - loss: 1.9362 - accuracy: 0.4377 - val_loss: 2.1442 - val_accuracy: 0.4307
Epoch 7/10
34/34 [==============================] - 1s 24ms/step - loss: 1.9314 - accuracy: 0.4508 - val_loss: 2.1928 - val_accuracy: 0.3933
Epoch 8/10
3

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming you have a dataset X_train and y_train
# X_train should be a 2D array with shape (number_of_samples, input_size)
# y_train should be a 2D array with shape (number_of_samples, output_size)

# Define the architecture parameters
input_size = 25  # Size of input layer
hidden_size = 100  # Size of hidden layers
output_size = 25  # Size of output layer

# Generate synthetic data (replace this with your actual data loading)
X_train = np.random.randn(1000, input_size)
y_train = np.random.randn(1000, output_size)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the input data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(input_size,)),
    tf.keras.layers.Dense(hidden_size, activation='relu', use_bias=False),
    tf.keras.layers.Dense(hidden_size, activation='relu', use_bias=False),
    tf.keras.layers.Dense(output_size, activation=None, use_bias=False)
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Print the model summary
model.summary()

# Train the model
epochs = 10
batch_size = 32
history = model.fit(X_train_scaled, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_scaled, y_test))


# Evaluate the model on the test set
_, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

# Print the combined accuracy result
print(f"Combined Accuracy on Test Set: {test_accuracy * 100:.2f}%")

Model: "sequential_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 100)               2500      
                                                                 
 dense_47 (Dense)            (None, 100)               10000     
                                                                 
 dense_48 (Dense)            (None, 25)                2500      
                                                                 
Total params: 15000 (58.59 KB)
Trainable params: 15000 (58.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
25/25 [==============================] - 1s 10ms/step - loss: 1.0633 - mae: 0.8218 - val_loss: 1.0122 - val_mae: 0.8002
Epoch 2/10
25/25 [==============================] - 0s 4ms/step - loss: 0.9993 - mae: 0.7971 - val_loss: 1.0012 - val_mae: 0.7953
Epoch 3/10
25/25 [==

In [ ]:
num_frames = 100  # Replace with the actual number of frames in your data
num_mfcc_features = 13  # Replace with the actual number of MFCC features extracted per frame
num_short_term_features = 20  # Replace with the actual number of short-term features in your data
num_classes = 15

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, LSTM, Dense, concatenate, Bidirectional, GRU, Reshape

# Define the input shape for MFCC features
mfcc_input = Input(shape=(num_frames, num_mfcc_features, 1), name='mfcc_input')

# CNN for processing MFCC features
conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(mfcc_input)
maxpool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
# Add more Conv2D and MaxPooling layers as needed

# Reshape the output of the CNN to a 3D tensor
reshaped_cnn_output = Reshape((-1, 32))(maxpool1)

# LSTM for extracting long-term features
lstm_output = LSTM(64)(reshaped_cnn_output)

# Define the input shape for short-term features
short_term_input = Input(shape=(num_frames, num_short_term_features), name='short_term_input')

# RNN for processing short-term features
rnn_output = Bidirectional(GRU(32))(short_term_input)

# Concatenate LSTM and RNN outputs
merged_output = concatenate([lstm_output, rnn_output])

# DNN for processing long-term features
dnn_output = Dense(64, activation='relu')(merged_output)
# Add more Dense layers as needed

# Probabilistic fusion algorithm (example: simple concatenation)
fusion_output = concatenate([lstm_output, dnn_output])

# Reshape the fusion_output to a 3D tensor
reshaped_fusion_output = Reshape((1, -1))(fusion_output)

# RNN for combining results
final_rnn_output = GRU(32, return_sequences=True)(reshaped_fusion_output)

# Output layer
num_classes = 10  # Replace with the actual number of classes
output = Dense(num_classes, activation='softmax')(final_rnn_output)

# Create the model
model = Model(inputs=[mfcc_input, short_term_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 mfcc_input (InputLayer)     [(None, 100, 13, 1)]         0         []                            
                                                                                                  
 conv2d_7 (Conv2D)           (None, 98, 11, 32)           320       ['mfcc_input[0][0]']          
                                                                                                  
 max_pooling2d_7 (MaxPoolin  (None, 49, 5, 32)            0         ['conv2d_7[0][0]']            
 g2D)                                                                                             
                                                                                                  
 reshape_7 (Reshape)         (None, 245, 32)              0         ['max_pooling2d_7[0][0]'